<a href="https://colab.research.google.com/github/ranjani277/BERT-Text-Classification/blob/main/Bert_Text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [1]:
!pip install transformers

In [1]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer,create_optimizer
import numpy as np
from sklearn.metrics import precision_recall_fscore_support,accuracy_score



In [2]:
import pandas as pd
data = pd.read_excel('/content/IMDB_Dataset_sample.xlsx')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
data['sentiment'] = data['sentiment'].map({'positive':1,'negative':0})

In [7]:
data_sample = data[0:100]

In [8]:
data_sample.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [9]:
data_sample.shape

(100, 2)

In [ ]:
#!pip uninstall -y pyarrow
#!pip install pyarrow

Found existing installation: pyarrow 17.0.0
Uninstalling pyarrow-17.0.0:
  Successfully uninstalled pyarrow-17.0.0
  Using cached pyarrow-17.0.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
Using cached pyarrow-17.0.0-cp310-cp310-manylinux_2_28_x86_64.whl (39.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [10]:

# convert pandas dataframe into huggingface dataframe
from datasets import Dataset
dataset = Dataset.from_pandas(data_sample)
dataset

Dataset({
    features: ['review', 'sentiment'],
    num_rows: 100
})

In [11]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
# tokenize the text inputs
def tokenize_function(example):
    return tokenizer(example["review"], truncation=True,padding="max_length")


# example means data , in the data we are using 'review' , while we are tokenize, we are using padding and truncation
# padding : i want to add zeros till max lenth
# truncate mean sentence cut : if my sentence more than specified tokens do not consider

In [13]:
tokenized_datasets = dataset.map(tokenize_function,batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

batched ==means instead of all the data at a time
we are sending batch wise
Q&A : how many samples in each batch here?


In [14]:
print(tokenized_datasets)

Dataset({
    features: ['review', 'sentiment', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 100
})


In [15]:
tokenized_datasets['review']

["One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the f

In [16]:
len(tokenized_datasets['input_ids'])

100

In [17]:
tokenized_datasets['input_ids'][0]

[101,
 2028,
 1997,
 1996,
 2060,
 15814,
 2038,
 3855,
 2008,
 2044,
 3666,
 2074,
 1015,
 11472,
 2792,
 2017,
 1005,
 2222,
 2022,
 13322,
 1012,
 2027,
 2024,
 2157,
 1010,
 2004,
 2023,
 2003,
 3599,
 2054,
 3047,
 2007,
 2033,
 1012,
 1026,
 7987,
 1013,
 1028,
 1026,
 7987,
 1013,
 1028,
 1996,
 2034,
 2518,
 2008,
 4930,
 2033,
 2055,
 11472,
 2001,
 2049,
 24083,
 1998,
 4895,
 10258,
 2378,
 8450,
 5019,
 1997,
 4808,
 1010,
 2029,
 2275,
 1999,
 2157,
 2013,
 1996,
 2773,
 2175,
 1012,
 3404,
 2033,
 1010,
 2023,
 2003,
 2025,
 1037,
 2265,
 2005,
 1996,
 8143,
 18627,
 2030,
 5199,
 3593,
 1012,
 2023,
 2265,
 8005,
 2053,
 17957,
 2007,
 12362,
 2000,
 5850,
 1010,
 3348,
 2030,
 4808,
 1012,
 2049,
 2003,
 13076,
 1010,
 1999,
 1996,
 4438,
 2224,
 1997,
 1996,
 2773,
 1012,
 1026,
 7987,
 1013,
 1028,
 1026,
 7987,
 1013,
 1028,
 2009,
 2003,
 2170,
 11472,
 2004,
 2008,
 2003,
 1996,
 8367,
 2445,
 2000,
 1996,
 17411,
 4555,
 3036,
 2110,
 7279,
 4221,
 12380,
 2854,
 

In [18]:
len(tokenized_datasets['input_ids'][0])

512

In [19]:
len(tokenized_datasets['input_ids'][1])

512

In [20]:
tokenized_datasets['attention_mask']

[[1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,


In [21]:
split_datasets = tokenized_datasets.train_test_split(test_size=0.2)

In [22]:
split_datasets

DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 80
    })
    test: Dataset({
        features: ['review', 'sentiment', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 20
    })
})

In [23]:
train_dataset = split_datasets['train'].to_tf_dataset()
train_dataset

<_PrefetchDataset element_spec={'review': TensorSpec(shape=(), dtype=tf.string, name=None), 'sentiment': TensorSpec(shape=(), dtype=tf.int64, name=None), 'input_ids': TensorSpec(shape=(512,), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(512,), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(512,), dtype=tf.int64, name=None)}>

In [27]:
train_dataset1 = split_datasets['train'].to_tf_dataset(columns=
                                                       ['input_ids','attention_mask'],
                                                       label_cols=['sentiment'],
                                                       shuffle=True,batch_size=2)
train_dataset1
#shuffle for train,batch size for training

<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [28]:
eval_dataset = split_datasets['test'].to_tf_dataset(
    columns=['input_ids','attention_mask'], label_cols=['sentiment'],shuffle=False, batch_size=2
         )
eval_dataset
# no shuffle for evaluation

<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [29]:
train_dataset1.cardinality().numpy()

# cardinality mean coming batch wise

40

In [30]:
eval_dataset.cardinality().numpy()

10

In [31]:
model = TFBertForSequenceClassification.from_pretrained(model_name,num_labels=2)
model

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0 (unused)
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
num_epochs =2
steps_per_epoch = len(train_dataset1) # 891 steps
num_train_steps = steps_per_epoch * num_epochs # 891*2
optimizer, schedule = create_optimizer(init_lr=2e-5, # adam
                                       num_warmup_steps=0, num_train_steps=num_train_steps)

In [34]:
optimizer
schedule

In [35]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]

In [36]:
model.compile(optimizer=optimizer,loss=loss_fn,metrics=metrics)

In [37]:
history = model.fit(train_dataset1,validation_data=eval_dataset,epochs=num_epochs,verbose=True)

Epoch 1/2
40/40 [==============================] - 60s 388ms/step - loss: 0.6659 - sparse_categorical_accuracy: 0.5625 - val_loss: 0.6521 - val_sparse_categorical_accuracy: 0.5500
Epoch 2/2
40/40 [==============================] - 12s 303ms/step - loss: 0.5421 - sparse_categorical_accuracy: 0.8250 - val_loss: 0.6378 - val_sparse_categorical_accuracy: 0.6500


In [38]:
history

In [40]:
# Evaluate the model
eval_results = model.evaluate(eval_dataset)
print(f"Evaluation results: {eval_results}")

10/10 [==============================] - 1s 123ms/step - loss: 0.6378 - sparse_categorical_accuracy: 0.6500
Evaluation results: [0.6377883553504944, 0.6499999761581421]


In [41]:
# Get predictions
predictions = model.predict(eval_dataset)
logits = predictions['logits']
pred_labels = tf.argmax(logits, axis=-1).numpy()

10/10 [==============================] - 1s 96ms/step


In [44]:
# Get true labels
true_labels = []
for x,y in eval_dataset:
  true_labels.extend(y.numpy()) # Extract true labels from the dataset

In [51]:
true_labels

[1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1]

In [45]:

# Compute metrics
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average='binary')
accuracy = accuracy_score(true_labels, pred_labels)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.65
Precision: 0.6666666666666666
Recall: 0.6
F1 Score: 0.631578947368421


In [46]:
# Save the model
model.save('checkpoints/my_bert_model')

In [47]:
# Load the model
loaded_model = tf.keras.models.load_model('checkpoints/my_bert_model')

Test with unseen data

In [48]:
from transformers import BertTokenizer
import numpy as np

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define a test sample
test_sample = "I love this product! It's amazing."

# Tokenize the test sample
inputs = tokenizer(test_sample, return_tensors='tf', padding='max_length', truncation=True, max_length=128)

# Print the tokenized inputs for verification
print("Tokenized Inputs:", inputs)

Tokenized Inputs: {'input_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
array([[ 101, 1045, 2293, 2023, 4031,  999, 2009, 1005, 1055, 6429, 1012,
         102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1,

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [50]:
# Make predictions
predictions = loaded_model(inputs)

# Extract logits and compute predicted class
logits = predictions['logits']
predicted_class = np.argmax(logits.numpy(), axis=-1)

# Print the prediction
print("Predicted Class:", predicted_class)


Predicted Class: [1]
